In [1]:
%AddJar file:///home/jovyan/data/magpie/dist/Magpie.jar

Starting download from file:///home/jovyan/data/magpie/dist/Magpie.jar
Finished download of Magpie.jar


# Modeling Glass-Forming Ability
This notebook builds and tests and machine learning model that predicts the glass-forming of metallic glass alloys. We start with creating a model and testing it with 10-fold, stratified cross-validation and then test it by withhold all of the data from the same alloy system when creating a test set.

In [2]:
import magpie.data.materials.CompositionDataset;
import magpie.data.materials.util.PropertyLists;
import magpie.data.utilities.modifiers.{ClassEliminationModifier, DiscreteToContinuousModifier};
import magpie.data.utilities.filters.PhaseDiagramExclusionFilter;
import magpie.data.utilities.splitters.AllMetalsSplitter;
import magpie.data.utilities.generators.PhaseDiagramCompositionEntryGenerator;
import magpie.models.BaseModel;
import magpie.models.classification.{SplitClassifier, WekaClassifier};
import magpie.utility.tools.PhaseDiagramExclusionValidator;
import scala.collection.JavaConversions._;

## Load in the Data
Read in the data from `datastes/glass.data`, and change the labels of all entries labeled 'partially crystalline' (AC) to 'crystalline' (CR)

In [3]:
val data = new CompositionDataset();

In [4]:
data.importText("../datasets/glass.data", null)
println(s"Read ${data.NEntries} entries.")

Read 5369 entries.


In [5]:
data.setTargetProperty("gfa", true)

Save the data for later plotting

In [6]:
data.saveCommand("training_data", "comp");

training_data.csv

Eliminate partially-crystalline labels

In [7]:
val mdfr = new ClassEliminationModifier();

In [8]:
mdfr.setClassToEliminate("AC")
mdfr.setNewClassName("CR");
mdfr.transform(data);

In [9]:
println(data)

Number of entries:    5369
Number of attributes:   0
Number of properties: 1
Target property: gfa
	Possible classes: {AM, CR}


## Compute Representation
Compute the 145 'general-purpose' features

In [10]:
data.setDataDirectory("/home/jovyan/data/magpie/Lookup Data")

In [11]:
for (prop <- PropertyLists.getPropertySet("general")) {
    data.addElementalProperty(prop);
}

In [12]:
data.generateAttributes()
println(s"Generated ${data.NAttributes} attributes")

Generated 145 attributes


## Train Machine Learning Model
We train two separate machine learning models: one for glasses that contain only metallic elements, and another for glasses that contain at least one nonmetallic element.

In [13]:
val model = new SplitClassifier()

In [14]:
model.setPartitioner(new AllMetalsSplitter())

In [15]:
val submodel = new WekaClassifier("meta.RandomSubSpace", 
    "-P 0.5 -S 1 -I 20 -W weka.classifiers.trees.RandomForest -- -I 10 -K 0 -S 1".split(" "))

In [16]:
model.setGenericModel(submodel)

In [17]:
model.train(data)

Save the model and dataset for later use

In [18]:
model.saveState("gfa-model.obj")

In [19]:
data.saveState("gfa-dataset.obj")

## Perform a 10-fold Crossvalidation
Run a simple cross-validation test: 10-fold, stratified cross-validation

### Using the Hierarchical Model

In [20]:
val cv_data = model.crossValidate(10, data)

In [21]:
println(model.ValidationStats)

Number Tested: 5369
Number correct: 4831 (89.980%)
Kappa: 0.6569
Sensitivity: 0.9088
FPR: 0.1263
Accuracy: 0.8998
PPV: 0.9542
NPV: 0.7679
FDR: 0.0458
MCC: 0.7517
F1: 0.9310
ROC AUC: 0.9131



### Using the Single Model

In [22]:
val cv_data = submodel.crossValidate(10, data)

In [23]:
println(submodel.ValidationStats)

Number Tested: 5369
Number correct: 4820 (89.775%)
Kappa: 0.6499
Sensitivity: 0.9059
FPR: 0.1262
Accuracy: 0.8977
PPV: 0.9547
NPV: 0.7596
FDR: 0.0453
MCC: 0.7463
F1: 0.9297
ROC AUC: 0.9170



The accuracies are similar to what is reported in the paper. And, we note that each the partitioned model yields identical performance to a single model trained on the entire dataset.

## Leave a Single System Out
In practice, we want to use this machine learning model to predict the glass-forming ability in alloy systems for which we have no training data. A k-fold crossvalidation test does not sufficient model this use case because we often have data from the same system in our training and test set. So, to test the ability of our model, we perform a test where we exclude all data from a single ternary system - Al-Ni-Zr - from our training set, train the model on the remaining data, and the assess the performance of our model on the Al-Ni-Zr data that was held-out.

In [24]:
def test_on_holdout_system(model : BaseModel, elements : Array[String]) : BaseModel = {
    // Generate training and test sets
    val trainData = data.clone();
    
    val filter = new PhaseDiagramExclusionFilter();
    filter.setElementList(elements)

    filter.setExclude(true);
    filter.filter(trainData)

    val testData = data.clone();
    filter.setExclude(false);
    filter.filter(testData)

    // Retrain the model
    val my_model = model.clone();
    my_model.train(trainData);
    
    my_model.externallyValidate(testData)
    return my_model
}

In [25]:
println(test_on_holdout_system(model, Array[String]("Al","Ni","Zr")).ValidationStats)

Number Tested: 207
Number correct: 181 (87.440%)
Kappa: 0.3953
Sensitivity: 0.8750
FPR: 0.1304
Accuracy: 0.8744
PPV: 0.9817
NPV: 0.4651
FDR: 0.0183
MCC: 0.5768
F1: 0.9253
ROC AUC: 0.6875



In [26]:
println(test_on_holdout_system(submodel, Array[String]("Al","Ni","Zr")).ValidationStats)

Number Tested: 207
Number correct: 175 (84.541%)
Kappa: 0.2558
Sensitivity: 0.8402
FPR: 0.0769
Accuracy: 0.8454
PPV: 0.9939
NPV: 0.2791
FDR: 0.0061
MCC: 0.4565
F1: 0.9106
ROC AUC: 0.7236



We find that both our hierarchical model and the single model perform decently on this dataset: an FPR ~ 10% and an area under the ROC curve of >0.6 in both cases. It appears our heirarchical model preforms better for this application, but this is just a single system.

### Visualize the Al-Ni-Zr predictions
To get a better idea of the performnace of the model, we will train the model on the dataset without Al-Ni-Zr data and then predict the glass-forming ability for a range of compositions in the system

In [27]:
val dataGen = new PhaseDiagramCompositionEntryGenerator();

In [28]:
dataGen.setElementsByName(Set[String]("Al", "Ni", "Zr"))
dataGen.setEvenSpacing(true)
dataGen.setOrder(1, 3)
dataGen.setSize(101)

In [29]:
val runData = data.emptyClone();
dataGen.addEntriesToDataset(runData);
println(s"Generated ${runData.NEntries} entries");

Generated 5151 entries


In [30]:
val filteredModel = test_on_holdout_system(model, Array[String]("Al","Ni","Zr"))

In [31]:
runData.generateAttributes();
filteredModel.run(runData)

Save the probabilities in composition format

In [32]:
val mdfr = new DiscreteToContinuousModifier();
mdfr.setClassName("AM")
mdfr.transform(runData)

In [33]:
val filename = runData.saveCommand("alnizr_predictions", "comp")
println(s"Saved data to ${filename}")

Saved data to alnizr_predictions.csv


## Perform a Leave-out-element-pairs Cross-validation Test
In this test, we iteratively without entires that contain a certain pair of elements for a test set. This is a generalized version of the "leaving out single system" test we just performed.

In [34]:
val validator = new PhaseDiagramExclusionValidator();
validator.setNElements(2)

Run the Hierarchical Model

In [35]:
validator.evaluateModel(model, data)
println(validator.printCommand(Seq[String]("last", "stats")))

Number Tested: 15318
Number correct: 12356 (80.663%)
Kappa: 0.3306
Sensitivity: 0.8438
FPR: 0.3067
Accuracy: 0.8066
PPV: 0.8934
NPV: 0.5930
FDR: 0.1066
MCC: 0.5111
F1: 0.8679
ROC AUC: 0.4864



Run the Single Model

In [36]:
validator.evaluateModel(submodel, data)
println(validator.printCommand(Seq[String]("last", "stats")))

Number Tested: 15318
Number correct: 12331 (80.500%)
Kappa: 0.3250
Sensitivity: 0.8439
FPR: 0.3119
Accuracy: 0.8050
PPV: 0.8906
NPV: 0.5944
FDR: 0.1094
MCC: 0.5079
F1: 0.8666
ROC AUC: 0.4849



We find very similar results to the paper. The perdiction accuracy is ~80% (better than random) and the 'hierarchical' model is *slightly* better for this test than the single model.